In [2]:
import spotipy

from spotipy.oauth2 import SpotifyClientCredentials

In [3]:
import pandas as pd
import requests

In [ ]:
# Coloca tus credenciales aquí
CLIENT_ID = ''
CLIENT_SECRET = ''
# Autenticación
auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [ ]:
# Coloca tus credenciales de lastfm aquí
SHARED_SECRET = ''
API_KEY = ''

last_fm_connect = requests.get(f"http://www.last.fm/api/auth/?api_key={API_KEY}")

In [10]:
paises_mediterraneos = {
    "ES": "España",
    "FR": "Francia",
    "MC": "Mónaco",
    "IT": "Italia",
    "SI": "Eslovenia",
    "HR": "Croacia",
    "BA": "Bosnia y Herzegovina",
    "ME": "Montenegro",
    "AL": "Albania",
    "GR": "Grecia",
    "MT": "Malta",
    "CY": "Chipre"}

In [11]:
def extraccion_por_paises(diccionario_zonas, años):
    todos_los_datos = [] # Esto es una lista de dicts donde se van guardando los diccionarios de cada país
    for pais in diccionario_zonas.keys():
        datos_rango = sp.search(q = f'year:{años}', limit=50, offset=0, type='track', market={pais}) # Cada una tiene que cambiar en q el rango de años
        datos_por_pais = {}
        datos_por_pais['pais'] = pais
        datos_por_pais['datos'] = datos_rango
        todos_los_datos.append(datos_por_pais)
    return todos_los_datos
   

In [12]:
def filtro_de_datos(lista_datos_zonas, datos):
    for datos_pais in lista_datos_zonas:
        for element in datos_pais['datos']['tracks']['items']:
            datos['pais'].append(datos_pais['pais'])
            datos['popularity'].append(element['popularity'])
            datos['nombre'].append(element['name'])
            artistas = []
            for artista in element['artists']:
                artistas.append(artista['name'])   
            artistas_str = ', '.join(artistas)
            datos['artista'].append(artistas_str)

In [13]:
p_mediterraneos = extraccion_por_paises(paises_mediterraneos, '2015-2019')

In [14]:
dic_mediterraneos = {'nombre': [], 'popularity': [], 'artista': [], 'pais': [], 'genero': []}

In [15]:
filtro_de_datos(p_mediterraneos, dic_mediterraneos)

In [16]:
dic_mediterraneos

{'nombre': ['El Efecto - Remix',
  "Greenwood, Philip Selway, Edward O'Brien, Colin Greenwood: Pyramid Song (Arr. Lawson)",
  'Amanece',
  'Enamorado',
  'Hasta Que Me Muera',
  'Holding Back the Years (Live at Ziggo Dome, Amsterdam)',
  'Fiebre (Prod. King DouDou)',
  'Слэмятся пацаны',
  'No Me Conoce - Remix',
  "I'm Better (feat. Lamb)",
  'LA CANCIÓN',
  'Stretch',
  'Cuatro Babys (feat. Trap Capos, Noriel, Bryant Myers & Juhn)',
  'Say You Love Me (Live at Ziggo Dome, Amsterdam)',
  'Sola (Remix)',
  'So Beautiful (Live at Ziggo Dome, Amsterdam)',
  'Borro Cassette',
  'A New Flame (Live at Ziggo Dome, Amsterdam)',
  'A Mí',
  'For Your Babies (Live at Ziggo Dome, Amsterdam)',
  'Madura',
  'Your Mirror (Live at Ziggo Dome, Amsterdam)',
  'Antes de Morirme (feat. ROSALÍA)',
  'Wenn jetzt Sommer wär (feat. Chapeau Claque)',
  'Esclava - Remix',
  'Someday in My Life (Live at Ziggo Dome, Amsterdam)',
  'Se Preparó',
  'Did You Love Me',
  'Diles',
  'Even Angels (feat. Blas Cantó)'

Llamada a API Last.FM y filtro para conseguir el género

In [ ]:
p_mediterraneos_last = requests.get(f"https://ws.audioscrobbler.com/2.0/?method=track.gettoptags&artist=Sasha&track=here+she+comes+again&api_key={API_KEY}&autocorrect=1&format=json")

In [18]:
p_mediterraneos_last.json()['toptags']['tag'][0]['name']

'pop'

In [20]:
lista_nombres_mediterraneo = list(dic_mediterraneos.values())[0]
lista_artistas_mediterraneo = list(dic_mediterraneos.values())[2]

In [21]:
def llamada_lastfm(lista_nombres, lista_artistas, datos):
    for i in range(len(lista_nombres)):
        nombre = str(lista_nombres[i]).replace(' ', '+')
        artista = lista_artistas[i].replace(' ', '+').replace(',', '+&')
        llamada_genero = requests.get(f"https://ws.audioscrobbler.com/2.0/?method=track.gettoptags&artist={artista}&track={nombre}&api_key={API_KEY}&autocorrect=1&format=json")
        try:
            genero_cancion = llamada_genero.json()['toptags']['tag'][0]['name']
            datos['genero'].append(genero_cancion)
        except KeyError:
            datos['genero'].append("NULL")
            print(f"La canción {lista_nombres[i]} no está registrada en lastfm")
            
        except IndexError:
            datos['genero'].append("NULL")
            print(f"La canción {lista_nombres[i]} no tiene tags")

In [22]:
llamada_lastfm(lista_nombres_mediterraneo, lista_artistas_mediterraneo, dic_mediterraneos)

La canción El Efecto - Remix no está registrada en lastfm
La canción Greenwood, Philip Selway, Edward O'Brien, Colin Greenwood: Pyramid Song (Arr. Lawson) no está registrada en lastfm
La canción Amanece no está registrada en lastfm
La canción Hasta Que Me Muera no tiene tags
La canción Holding Back the Years (Live at Ziggo Dome, Amsterdam) no tiene tags
La canción Слэмятся пацаны no está registrada en lastfm
La canción No Me Conoce - Remix no está registrada en lastfm
La canción I'm Better (feat. Lamb) no está registrada en lastfm
La canción LA CANCIÓN no está registrada en lastfm
La canción Cuatro Babys (feat. Trap Capos, Noriel, Bryant Myers & Juhn) no está registrada en lastfm
La canción Say You Love Me (Live at Ziggo Dome, Amsterdam) no tiene tags
La canción Sola (Remix) no está registrada en lastfm
La canción So Beautiful (Live at Ziggo Dome, Amsterdam) no tiene tags
La canción A New Flame (Live at Ziggo Dome, Amsterdam) no tiene tags
La canción For Your Babies (Live at Ziggo Dome

In [23]:
df_mediterraneos = pd.DataFrame(dic_mediterraneos)

In [24]:
df_mediterraneos.index.name="id"

In [25]:
df_mediterraneos

,nombre,popularity,artista,pais,genero
id,,,,,
0,El Efecto - Remix,68,"Rauw Alejandro, Chencho Corleone, KEVVO, Bryan...",ES,NULL
1,"Greenwood, Philip Selway, Edward O'Brien, Coli...",3,"Jonny Greenwood, Thom Yorke, Philip Selway, Ed...",ES,NULL
2,Amanece,75,"Anuel AA, Haze",ES,NULL
3,Enamorado,2,Alexander Stewart,ES,pop
4,Hasta Que Me Muera,70,Bryant Myers,ES,NULL
...,...,...,...,...,...
595,Die Heelal (feat. Early B),0,"Janie Bay, Early B",CY,NULL
596,GANGSTA,48,"SNIK, A!MS",CY,NULL
597,Sirenen,0,Tommy,CY,NULL


In [27]:
df_mediterraneos.to_csv("paises_mediterraneros_2015_2019.csv")